In [3]:
#liberias
import pandas as pd
import numpy as np

# Carga y Exploración Datos

In [5]:
path='../data/data.csv'
df=pd.read_csv(path)
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385,80,1,195,NaN
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052,50,1,360,5.0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717,30,0,407,3.0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606,30,0,544,4.0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346,30,1,170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424,30,1,106,NaN
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514,50,0,257,NaN
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385,80,1,347,5.0
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974,30,0,582,1.0


## Valores Nulos

In [ ]:
nulos = df.isnull().sum()
porcentaje_nulos = (nulos / len(df)) * 100
tabla_nulos = pd.DataFrame({
    'Nulos': nulos,
    '% Nulos': porcentaje_nulos.round(1)
})
tabla_nulos[tabla_nulos['Nulos'] > 0]

,Nulos,% Nulos
Casado,190,1.8
Graduado,103,1.0
Campanna_anno,5707,52.8


**Tratamiento de los nulos**
- `Casados` y `Graduados`: los eliminamos porque son únicamente un 2%(pero a futuro).
- `Campanna_años`:  Vamos a sustituir los valores faltantes de 0.

**Casadas y Graduados**

In [8]:
df = df.drop(columns=['Casado', 'Graduado'])

**Campanna_años**

In [9]:
df['Campanna_anno'] = df['Campanna_anno'].fillna(0)